# Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pylab
from pprint import pprint
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("AIS_Notebook")

# Load Signal

In [ ]:
# load signal
import os
from py_audiolib.io_tools import wavefile
# dir0 = os.environ["DATA_FILES"]
# file0 = dir0 + "/communications/AIS IQ.wav"
file0 = "./SDRuno_20200907_184926Z_161985kHz.wav"
tmp_signal, info = wavefile.read_from_file(file0, float, False)
signal = tmp_signal[:,0] + 1j*tmp_signal[:,1]
pylab.specgram(signal, Fs=info["sample_rate"])
pylab.show()

# apply burst detection
from kcss.detectors.detector_energy import DetectorEnergy
det_energy = DetectorEnergy(100)
[bursts, sample_loc] = det_energy.analyze_signal(signal, True)
pprint(sample_loc)

# AIS
Automatic Identification System (AIS)

# Modulation
| Characteristic | Value |
| :-: | :-: |
| Frequency | 161.975 MHz and 162.025 MHz |
| Modulation | GMSK |
| Baud | 9600 symbols/s |

# Data Protocol

* The messages are HDLC encoded (with bit stuffing)
* Each message starts with a preamble of 24 bits of (0101...01)
* The bits are further NRZI encoded.

| Preamble | SB | Data | CRC | SB |
| :-: | :-:| :-: | :-: | :-: |
| 010101010101010101010101 | 01111110 | Bit stuffed data payload | 16 bit CRC-16-CCITT | 01111110 |





In [ ]:
from kcss.receivers.burst_receiver import BurstReceiver
from kcss.line_codes import line_code
from kcss.line_codes.hdlc import HDLC
from kcss.utils.binary import Binary
from kcss.utils.converter import bits_to_str

HDLC_FLAG = [0,1,1,1,1,1,1,0]
START_FLAG = np.array([0,1]*0 + HDLC_FLAG)

# prepare low pass filter
from scipy.signal import lfilter,firwin
lpf = firwin(31, 10e3, fs=info["sample_rate"])


# Prepare FSK Demodulator
# NOTE: spec = {"baud": 9600.0, "symbol_map":[0, 1], "mod_index":0.5}
receiver = BurstReceiver()
fs = info["sample_rate"]
spec = {"baud": 9600.0, "symbol_map":[1, 0], "mod_index":0.5}


# prepare output and track burst
msgs = [None] * len(bursts)
burst_count = 0

for burst in bursts:
    logger.info("\n" + "="*50 + "\n" + "Burst = %d" % burst_count)
    burst_count += 1

    # NOTE: shift the burst in frequency 
    #       specific to SDRuno_20200907_184926Z_161985kHz.wav file
    c_burst = burst * np.exp(1j*2*np.pi*11e3*np.arange(len(burst))/fs)
    c_burst = lfilter(lpf, 1, c_burst)

    # c_burst = burst
    plt.subplot(311)
    plt.plot(abs(c_burst))
    plt.subplot(312)
    plt.plot(np.angle(c_burst));
    plt.subplot(313)
    plt.psd(c_burst, Fs=fs); plt.title(f"Burst {burst_count}");
    plt.show()
    
    # demodulate GMSK
    c_demod_bits = receiver.fsk_receiver(c_burst, fs, spec)
    
    # NRZI decode
    c_bits = 1 - line_code.nrzi_decode(c_demod_bits)
    
    # ---------------  synchronize  ------------------
    # (search preamble + start flag, and end flag)
    start_ind = Binary.identify_location(
        c_bits, START_FLAG, threshold=len(START_FLAG))
    
    #extract last occurence of HDLC flag
    end_ind = Binary.identify_location(
        c_bits, HDLC_FLAG, threshold=len(HDLC_FLAG), first=False)

    if start_ind and end_ind:
        logger.info(f"Start = {start_ind} and End = {end_ind}, length = {end_ind - start_ind}")
        synced_bits = c_bits[start_ind + len(START_FLAG):end_ind]
        
    else:
        # NOTE: was not able to find either start or end HDLC flag
        logger.info(f"start={start_ind} and/or end={end_ind} flag missing\n\n")
        continue

    # HDLC deframer
    hdlc_frame = HDLC.bit_destuff(synced_bits)
    
    # ----------------  print info  -------------------
    
    logger.info(f"Message of length {len(hdlc_frame)} bits\n\t{bits_to_str(hdlc_frame)}\n")

    msgs[burst_count-1] = hdlc_frame

# Messages

In [ ]:
from kcss.error_codes import cyclic_redundancy_check as CRCMOD
CCITT = CRCMOD.STANDARDS["CRC-16-CCITT"]

# load AIS message protocols
from kcss.protocol.data_packet import DataPacket as DP
# AIS = DP.load_json("/home/keithchow/ais_message.json")
AIS = [
    {
        "name": "Position Report",
        "fields": [
            {"name": "Message_ID", "nbits": 6},
            {"name": "Repeat_Ind", "nbits": 2},
            {"name": "User_ID", "nbits":30, "dtype": "BCD"},
            {
                "name": "Navigational Status", "nbits":4, "dtype":"ENUM",
                "args": {
                    "msb_first": True,
                    "map": {
                        "0": "Under way using engine",
                        "1": "At Anchor",
                        "2": "Not under command",
                        "3": "Restricted Maneuverability",
                        "4": "Constrained by her draught",
                        "5": "Moored",
                        "6": "Aground",
                        "7": "Engaged in Fishing",
                        "8": "Under way, sailing",
                        "9": "Reserved for future amendment"
                    }
                }
            }
        ]
    }
]

from kcss.info import numeric_info as NI
for msg in msgs:
    # -------------  skip Nones  --------------------------
    if msg is None:
        continue
        
    #
    print("="*70 + "\nMessage length = %d"%len(msg))

    # FIXME: Why don't CRC match?
    est_crc = CRCMOD.crc(msg[:-16].astype(np.int8), zeropad=False, **CCITT)
    # est_crc = CRCMOD.crc(msg.astype(np.int8), **CCITT)
    print("CRC parity   = %s"%str(msg[-16:]))
    print("CRC estimate = %s"%str(NI.encode_bcd(est_crc, 16)))

    # print messages
    # print("\n" + AIS[0].to_str(msg[:AIS[0].nbits]))

# ChatGPT generated example for MSB-first input

* <font color=red>Failed to validate against this example</font>


In [ ]:
bits_0 = np.array([1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1])
expected_crc = "0010100110110001" # 0x29A1

est_crc = CRCMOD.crc(bits_0.astype(np.int8), zeropad=True, input_LSB=False, **CCITT)
print(est_crc)
print(hex(est_crc))

# Example speicific to AIS

* <font color=red>Failed to validate against this example</font>
* [reference](https://www.dsprelated.com/showthread/comp.dsp/79522-1.php)
  * ChatGPT had mentioned that AIS inputs on MSB side...so trying to identify examples to test against.


In [ ]:
#https://www.dsprelated.com/showthread/comp.dsp/79522-1.php
# NOTE: example showed HDLC Flags...should bit stuffing be removed?
bits_1 = np.array(
    [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,1,1,1,1,0,1,1,1,0,1,1,1,0,1,1,0,0,1,1,0,0,0,1,1,1,0,1,0,1,0,0,1,1,1,1,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,0,0,1,1,0,1,1,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0]
)
bits_1b = HDLC.bit_destuff(bits_1)
exp_crc = "1110100000011111"
est_crc = CRCMOD.crc(bits_1b, zeropad=True, input_LSB=True, **CCITT)

logger.info(f"Expected = {hex(int(exp_crc, 2))}, estimate = {hex(est_crc)}")

# Example CRC16-CCITT Example

* This example is from [CRC16-CCITT](https://srecord.sourceforge.net/crc16-ccitt.html#:~:text=Provides%20source%20code%20for%20the,adequate%20in%20a%20closed%20system.)
* This pushes in input bits at the LSB and shifts left.


In [ ]:

# https://srecord.sourceforge.net/crc16-ccitt.html#:~:text=Provides%20source%20code%20for%20the,adequate%20in%20a%20closed%20system.
bits_2 = np.array([0,1,0,0,0,0,0,1])
exp_crc = "1001010001111001" # 0x9479

# calculate the CRC of the input bits
est_crc = CRCMOD.crc(bits_2, zeropad=True, input_LSB=True, **CCITT)

logger.info(f"Expected = {hex(int(exp_crc, 2))}, estimate = {hex(est_crc)}")

if int(exp_crc, 2) == est_crc:
    logger.info("CRC Decode matches expected!!!")
else:
    logger.warning("Failed on CRC16-CCITT example")